# Super Lucky Palindromes Notebook

Original problem : https://www.spoj.com/problems/CTPLUCKY/

This notebook will present my work on the problem I had really fun solving for the past two months.
At first, I solved this problem with a naive algorithm (v1 and v2) but they were not fast enough to solve the 10^18th index that problem's statement wants.

I did a break of 2 months to came back with the idea of solving it. <br>
Which I did.

In this notebook you'll find the predata code which helps to get initial parameters for v1, v2 and v3. <br>
All algorithm version I did to solve the problem. <br>
At the end a time comparison between these algorithm which will suprise you I think :)

Little vocabulary :
- A lucky number is a number that is only made of "4" and "7"
- A super lucky number is a lucky number where either its amount of "4" or "7" is itself a lucky number
- A super lucky palindrome is a super lucky number which is also a palindrome
- Super Lucky Palindrome will be called SLP

## Predata

This predata code computes initial parameters to make a slightly faster algorithm.<br>
These values are always the same, that's why I can store them in array.

The statement says I only need to get index until 10^18th one.<br>
With combination formula I know how much possibilities exist with a given length number.

This code print 3 differents values :
- Possible numbers length which are (4, 7, 44, 47, 74, 77, 444). More is useless because the 10^18th indexis a 444 length number.
- Amount of possibilities for each length.
- Pair numbers. To know all possible amount of "4" and "7" one side of a palindrome can have to make a SLP (I call it Pair numbers because if you know the amount of "4" you can get the amount of "7", they work by pair)

It works because I "simulate" the complete palindrome number from only one side. So I need to check if the amount of "4" or either the amount of "7" will be a lucky number.

In [1]:
import scipy.special

def is_lucky(length):
    if any(x in str(length) for x in "01235689"):
        return 0
    return 1

def get_pair(length):
    pair = []
    limit = length // 2
    is_odd = length % 2
    
    for amount_4 in range(0, limit+1):
        if is_lucky(amount_4*2+is_odd) == 1 or is_lucky(amount_4*2) == 1 or \
        is_lucky(length-amount_4*2-is_odd) == 1 or is_lucky(length-amount_4*2) == 1:
            pair.append(amount_4)
    return pair

def get_possibilities(length, pair):
    total = 0
    for i in pair:
        total += scipy.special.comb(length, i, exact=True)
    return total

In [2]:
for i in range(4,445):
    if is_lucky(i) == 1:
        pair = get_pair(i)
        print(i, pair, int(get_possibilities(i//2, pair)))

4 [0, 2] 2
7 [0, 1, 2, 3] 8
44 [0, 2, 20, 22] 464
47 [0, 1, 2, 3, 20, 21, 22, 23] 4096
74 [0, 2, 15, 22, 35, 37] 18728400854
77 [0, 1, 2, 3, 15, 16, 22, 23, 35, 36, 37, 38] 75422540336
444 [0, 2, 22, 37, 185, 200, 220, 222] 3949534246514075237194788708873582383079842


## v1 : Naive algorithm

The idea is simple, create every possibilities and check for every possibility if its a SLP.<br>
The numbers I'm working with are palindromes which implies that I can work with only one side to get the other one. Once one side is calculated, I just have to reverse it and adding it to itself and it will makes a palindrome number.<br>

One little issue with this are odd numbers. Odd numbers have one more digit I need to find to create a SLP<br>
To do get the middle digit I use a condition that ask "if I create a palindrome number with the side I calculated and I add a "4" at the middle, is it a super lucky number ? Same with adding a "7" instead". Hopefully there can only be one good anwser.

I also discovered that once you've found half of the solutions for a given length, you just have to change the "4" by "7" and "7" by "4" to create the opposite solution. You can look at query 6 and 7, 5 and 8, 4 and 9 ... you'll see they are opposite<br>
This way I can optimize the algorithm even more with the help of "self.reverse" which will exchange the "4" by "7".

I also need to get the real query value before starting. Range_list stores the amount of possibilities for each length. I have to decrease the value of query by range_list values one by one until query gets negative. Once done, I'll know what is the length of the solution and what are the pair numbers implied.

About the recursive function :<br>
Instead of having an empty array and placing either a "4" or a "7" inside it I create an all "4" (or "7") array and I just change the values if I have to. This way I can create a more generic code.

In [3]:
import copy

class lucky_v1():
    def __init__(self): 
        self.length_list = [4, 7, 44, 47, 74, 77, 444]
        self.range_list = [2, 8, 464, 4096, 18728400854, 75422540336, 3949534246514075237194788708873582383079842]
        self.pair_list = [[0,2], [0,1,2,3], [0,2,20,22], [0,1,2,3,20,21,22,23], [0,2,15,22,35,37], [0,1,2,3,15,16,22,23,35,36,37,38]]
        
        self.index = 0
        self.query = 0
        self.limit = 0
        self.cursor = 1
        self.reverse = False
        self.solution = ""
    
    def get_solution(self, query):        
        self.index = 0
        while query - self.range_list[self.index] > 0:
            query -= self.range_list[self.index]
            self.index += 1
        
        self.reverse = False
        if query - (self.range_list[self.index]//2) > 0:
            query = self.range_list[self.index] - query + 1
            self.reverse = True
        
        self.query = query
        self.limit = self.length_list[self.index]//2
        self.cursor = 0
        self.solution = ""
        if self.reverse == False:
            self.generate(["4"]*self.limit) #Everything
        else:
            self.generate(["7"]*self.limit) #Everything
        
        return "".join(self.solution)
        
    def put(self, res, pos):
        if res[pos] == "4":
            res[pos] = "7"
        else:
            res[pos] = "4"
        return res
    
    def generate(self, res, pos=0, num4=0, num7=0):
        if self.cursor > self.query:
            return
        if pos == self.limit:
            if num4 in self.pair_list[self.index] or num7 in self.pair_list[self.index]:
                self.cursor += 1
                if self.cursor == self.query:
                    if self.length_list[self.index]%2 == 0:
                        self.solution = res + res[::-1]
                    else:
                        if num4*2 + 1 in self.length_list or num7*2 in self.length_list:
                            self.solution = res + ['4'] + res[::-1]
                        else:
                            self.solution = res + ['7'] + res[::-1]
                        if self.reverse == True:
                            self.put(self.solution, self.limit)
        else:
            self.generate(res, pos + 1, num4 + 1, num7)
            self.generate(self.put(copy.copy(res), pos), pos + 1, num4, num7 + 1)

In [4]:
v1 = lucky_v1()
for i in range(1,20):
    print(v1.get_solution(i), i)

4444 1
7777 2
4444444 3
4477744 4
4747474 5
4774774 6
7447447 7
7474747 8
7744477 9
7777777 10
44444444444444444444444444444444444444444444 11
44444444444444444444777744444444444444444444 12
44444444444444444447477474444444444444444444 13
44444444444444444447744774444444444444444444 14
44444444444444444474477447444444444444444444 15
44444444444444444474744747444444444444444444 16
44444444444444444477444477444444444444444444 17
44444444444444444744477444744444444444444444 18
44444444444444444744744744744444444444444444 19


## v2 : Naive algorithm - Optimized

The only thing that change with the previous version is the recursive part.

The previous algorithm has a big flaw because I'm computing every exiting possibilities and then I have to check if the current result can become a SLP.<br>
I created the function "is_possible" which can fix this issue by detecting as soon as possible if the current solution i'm computing will give a SLP or not.

The idea is simple : With the current state of the result, does a solution still exists if I add a "4" (or a "7") ?<br>
Refering to the pair numbers, I know how much "4" or "7" I need to have at the end.<br>
So if I don't have enough space left to write a "4" and create a SLP, it is useless to add a "4".

Thanks to this I'm sure that, once every digit is written, I do have an existing solution.<br>
I still have to compute the middle digit of odd length numbers tho.

In [5]:
import copy

class lucky_v2():
    def __init__(self):
        self.length_list = [4,7,44,47,74,77,444]
        self.range_list = [2, 8, 464, 4096, 18728400854, 75422540336, 3949534246514075237194788708873582383079842]
        self.pair_list = [[0,2], [0,1,2,3], [0,2,20,22], [0,1,2,3,20,21,22,23], [0,2,15,22,35,37], [0,1,2,3,15,16,22,23,35,36,37,38]]
        
        self.index = 0
        self.limit = 0
        self.query = 0
        self.cursor = 1
        self.reverse = False
        self.solution = ""
    
    def get_solution(self, query):
        self.index = 0
        while query - self.range_list[self.index] > 0:
            query -= self.range_list[self.index]
            self.index += 1
        
        self.reverse = False
        if query - (self.range_list[self.index]//2) > 0:
            query = self.range_list[self.index] - query + 1
            self.reverse = True
        
        self.limit = self.length_list[self.index]//2
        self.query = query
        self.cursor = 0
        self.solution = ""
        if self.reverse == False:
            self.generate(["4"]*self.limit) #Everything
        else:
            self.generate(["7"]*self.limit) #Everything
        return "".join(self.solution)
        
    def put(self, res, pos):
        if res[pos] == "4":
            res[pos] = "7"
        else:
            res[pos] = "4"
        return res
    
    def is_possible(self, amount, space):
        idx = 0
        while self.pair_list[self.index][idx] <= amount:
            idx += 1
        if self.pair_list[self.index][idx] <= amount + space:
            return 1
        return 0
    
    def generate(self, res, pos=0, num4=0, num7=0):
        if self.cursor > self.query:
            return
        if pos == self.limit:
            self.cursor += 1
            if self.cursor == self.query:
                if self.length_list[self.index]%2 == 0:
                    self.solution = res + res[::-1]
                else:
                    if num4*2 + 1 in self.length_list or num7*2 in self.length_list:
                        self.solution = res + ['4'] + res[::-1]
                    else:
                        self.solution = res + ['7'] + res[::-1]
                    if self.reverse == True:
                        self.put(self.solution, self.limit)
        else:
            if self.is_possible(num4, self.limit-pos) == 1:
                self.generate(res, pos + 1, num4 + 1, num7)
            if self.is_possible(num7, self.limit-pos) == 1:
                self.generate(self.put(copy.copy(res), pos), pos + 1, num4, num7 + 1)

In [6]:
v1 = lucky_v1()
v2 = lucky_v2()

for i in range(1,20):
    print(i)
    if v1.get_solution(i) != v2.get_solution(i):
        print(v1.get_solution(i))
        print(v2.get_solution(i))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


## v3 : Mathematical approach - Limited

I will not explain in detail every part of the code here (check the github) but just the idea in general.

The idea of this algorithm is like this :<br>
I know how much possibilities there is for a given length (refer you to predata code and range_list arrays) I can know how much length I need to have before putting the first "7".<br>
Exemple : Let's have this parameters, length = 4, pair = [0,2], query = 8. The possibilities are :<br>
- 0 > 4444
- 1 > 4477
- 2 > 4747
- 3 > 4774
- 4 > 7447
- 5 > 7474
- 6 > 7744

They are seven of them. The amount of these possibilities is calculated with binomial function (check predata for this).<br>
My query is currently 8, I am now sure that the first "7" to put will not be at one of these indexes.<br>
But, If I calculate with a length of 5 it gives this :<br>
- 0 > 44444
- 1 > 44477
- 2 > 44747
- 3 > 44774
- 4 > 47447
- 5 > 47474
- 6 > 47744
- 7 > 74447
- 8 > 74474
- 9 > 74744
- 10> 77444

The query 8 <= 10 so I am sure the first "7" to put will be contained in one of the last 4 solutions.<br>
So now I do a little trick, I decrease the value of query by the amount of possibilities from length-1. In others terms, I "remove" the possibilities which doesn't have a "7" at their last position (or first position depending of your vocabulary).<br>
Mind that we also need to decrease the pair values because we just put one "7". That means the pair[0] won't be possible anymore !<br>
And so we can continue with the same logic. Find how much possibilities exists for all length to know exactly when the next "7" will be placed at.

One last thing, this algorithm doesn't work with very large numbers. It had an issue I had trouble with.<br>
The algorithm was working perfectly for first indexes (until ~10^18), why suddenly it would stops working for big length number ???<br>
I found out why, it was the use of float numbers which are not precise enough for big length SLP.

That's why the v4 exists :)

Little note : the solution is also fast enough to skip the "reverse" feature

In [7]:
import copy
import scipy.special

class lucky_v3():
    def __init__(self):
        self.length_list = [4, 7, 44, 47, 74, 77, 444, 447, 474, 477, 744, 747, 774, 777]
        self.range_list = [2, 8, 464, 4096, 18728400854, 75422540336, 3949534246513022733692774166275217636196352, 27912724205136969367829202079257831965458432, 54901863838981370331031720163633347084419072, 408867219021824519558760381286789756865216512, 731060835093268059719935428380319894767978536241103340770302477220618973863638154274876419272070389261402112, 3028768728981208771138392332161427475712344495983372066026850069568299204087095837473068158977043396135223296, 383767495066681352637998366663200070705183074877734592607500475472627707939626649121930829298776144224727016144896, 1564887013153773591096691217957484965016329465244742414975904717557119142894155366820818937587853276546953480503296]
        self.pair_list = [[0,2], [0,1,2,3], [0,2,20,22], [0,1,2,3,20,21,22,23], [0,2,15,22,35,37], [0,1,2,3,15,16,22,23,35,36,37,38], [0,2,22,37,185,200,220,222], [0,1,2,3,22,23,37,38,185,186,200,201,220,221,222,223], [0,2,15,22,37,200,215,222,235,237], [0,1,2,3,15,16,22,23,37,38,200,201,215,216,222,223,235,236,237,238], [0,2,22,37,135,150,222,237,335,350,370,372], [0,1,2,3,22,23,37,38,135,136,150,151,222,223,237,238,335,336,350,351,370,371,372,373], [0,2,15,22,37,150,165,222,237,350,365,372,385,387], [0,1,2,3,15,16,22,23,37,38,150,151,165,166,222,223,237,238,350,351,365,366,372,373,385,386,387,388]]
        
        self.limit = 0
        self.seven_amount = 0
        self.pair = []
        self.solution = ""
    
    def get_solution(self, query):
        index = 0
        while query - self.range_list[index] > 0:
            query -= self.range_list[index]
            index += 1
        
        query -= 1
        self.limit = self.length_list[index]//2
        self.seven_amount = 0
        self.pair = copy.copy(self.pair_list[index])
        self.solution = ["4"] * self.limit
        
        self.slp_algorithm(query) #everything
        
        if self.length_list[index]%2 == 0:
            self.solution = self.solution[::-1] + self.solution
        else:
            if self.seven_amount*2+1 in self.length_list or (self.limit-self.seven_amount)*2 in self.length_list:
                self.solution = self.solution[::-1] + ["7"] + self.solution
            else:
                self.solution = self.solution[::-1] + ["4"] + self.solution
        
        return "".join(self.solution)

    def get_comb(self, length, offset=0):
        comb = []
        for pos in range(1, length+1):
            total = 0
            for i in self.pair:
                i -= offset
                if i <= pos:
                    total += scipy.special.comb(pos, i)
            comb.append(total)
        return comb
    
    def slp_algorithm(self, cursor, pos=0, depth=0):
        if self.pair[0] - depth < 0:
            del self.pair[0]
        if cursor < 0:
            return
        if cursor == 0:
            for i in range(self.pair[0] - depth):
                self.solution[i] = "7"
                self.seven_amount += 1
            return
        
        comb = self.get_comb(self.limit - pos, depth)
        idx = 0
        while idx < self.limit - depth and cursor >= comb[idx]:
            idx += 1
        self.solution[idx] = "7"
        self.seven_amount += 1
        cursor -= comb[idx-1]
        self.slp_algorithm(cursor, self.limit - idx, depth + 1)

In [8]:
yes = lucky_v3()
for i in range(1, 25):
    print(yes.get_solution(i), i)

4444 1
7777 2
4444444 3
4477744 4
4747474 5
4774774 6
7447447 7
7474747 8
7744477 9
7777777 10
44444444444444444444444444444444444444444444 11
44444444444444444444777744444444444444444444 12
44444444444444444447477474444444444444444444 13
44444444444444444447744774444444444444444444 14
44444444444444444474477447444444444444444444 15
44444444444444444474744747444444444444444444 16
44444444444444444477444477444444444444444444 17
44444444444444444744477444744444444444444444 18
44444444444444444744744744744444444444444444 19
44444444444444444747444474744444444444444444 20
44444444444444444774444447744444444444444444 21
44444444444444447444477444474444444444444444 22
44444444444444447444744744474444444444444444 23
44444444444444447447444474474444444444444444 24


## v4 Mathematical approach - No limit (Non recursive)

It is poorly coded (imo) but the algorithm do the job very well. <br>
I have not really anything to say.

This v4 is basically v3 using only python integers and with a non recursive function to solve the problem.<br>
That means you can compute query 10^100 it will works

What ?<br>
Yeah, 10^100<br>
You wanna try ? How much time do you think it takes ?<br>
Imo, I think it is not enough. I need to optimize until 10^10000

I have coded a verbose feature if you're curious about the inside work of the algorithm for very very large number

In [ ]:
import scipy.special
import time
from sys import stdout

class lucky_v4():
    def __init__(self, verbose=False):
        self.verbose = verbose
        self.time_elapsed = 0
        
        self.query = 0
        self.cursor = 0
        self.length = 0
        self.limit = 0
        self.pair = []
        self.seven_amount = 0
        self.total = 0
        self.solution = ""
        
    def verbose_print(self, text_index): #ignore this, it's just display
        if self.verbose == False:
            return
        
        if text_index == 0:
            self.time_elapsed = time.time()
            print("Algorithm started")
        if text_index == 1:
            stdout.write("\rSolution will have a length of %d. Current length of amount of possibilities is %d" % (self.length, len(str(self.total))))
        if text_index == 2:
            print("\n")
            print("Now computing solution for query", self.query)
            print()
        if text_index == 3:
            stdout.write("\rCursor currently at position %d/%d (%d%%). Amount of seven put %d. Time elapsed %f" \
                         % (self.limit-self.space, self.limit, int((self.limit-self.space)/self.limit*100), self.seven_amount, time.time() - self.time_elapsed))
        if text_index == 4:
            print("\n")
            print("Algorithm finished")
            print("Solution have a length of", len(self.solution), "and have", self.solution.count("4"), "'4' and", self.solution.count("7"), "'7'")
            print()
            print("Total time elapsed", time.time() - self.time_elapsed)
        
    def is_lucky(self, length):
        if any(x in str(length) for x in "01235689"):
            return 0
        return 1
    
    def get_palindrome_pair(self, length):
        pair = []
        is_odd = length % 2
        limit = length // 2
        total = 0
        for x in range(0, limit+1):
            if self.is_lucky(x*2+is_odd) == 1 or self.is_lucky(x*2) == 1 or \
            self.is_lucky(length-x*2-is_odd) == 1 or self.is_lucky(length-x*2) == 1:
                pair.append(x)
        return pair
    
    def get_lucky_from_index(self, index):
        binary = "{0:b}".format(index+2)
        res = 0
        for idx, i in enumerate(binary[1:]):
            res += (4 if i == "0" else 7) * 10**(len(binary)-idx-2)
        return res
        
    def get_solution(self, query):
        self.verbose_print(0)
        
        self.query = query
        index = 0
        while self.query > 0:
            self.total = 0
            self.length = self.get_lucky_from_index(index)
            self.pair = self.get_palindrome_pair(self.length)
            self.limit = self.length // 2
            for i in self.pair:
                self.total += scipy.special.comb(self.limit, i, exact=True)
            self.query -= self.total
            index += 1
            self.verbose_print(1)
        
        self.query += self.total - 1
        self.seven_amount = 0
        self.verbose_print(2)
        self.solution = self.slp_algorithm() #everything
        
        if self.length%2 == 0:
            self.solution = self.solution[::-1] + self.solution
        else:
            if self.is_lucky(self.seven_amount*2+1) == 1 or self.is_lucky((self.limit-self.seven_amount)*2) == 1:
                self.solution = self.solution[::-1] + "7" + self.solution
            else:
                self.solution = self.solution[::-1] + "4" + self.solution
                
        self.verbose_print(4)
        return self.solution
    
    def get_comb(self, offset=0):
        comb = []
        for pos in range(1, self.limit+1):
            total = 0
            for i in self.pair:
                i -= offset
                if i <= pos:
                    total += scipy.special.comb(pos, i, exact=True)
            comb.append(total)
        return comb
    
    def slp_algorithm(self):
        self.cursor = self.query
        self.space = self.limit
        self.solution = ["4"] * self.space
        amount = 0

        while self.cursor > 0:
            comb = self.get_comb(amount)
            idx = 0
            while idx < self.limit - self.seven_amount and self.cursor >= comb[idx]:
                idx += 1
            
            self.solution[idx] = "7"
            self.seven_amount += 1
            
            if idx <= 0:
                break
            self.cursor -= comb[idx-1]
            self.space = idx
            amount += 1
            self.verbose_print(3)
        
        if self.cursor == 0:
            idx = 0
            while self.pair[idx] < amount:
                idx += 1
            for i in range(self.pair[idx] - amount):
                self.solution[i] = "7"
                self.seven_amount += 1
        self.space = 0
        self.verbose_print(3)

        return "".join(self.solution)

In [ ]:
query = 2
lucky_v4(True).get_solution(10)

Algorithm started
Solution will have a length of 7. Current length of amount of possibilities is 1

Now computing solution for query 7

Cursor currently at position 3/3 (100%). Amount of seven put 3. Time elapsed 0.001999

Algorithm finished
Solution have a length of 7 and have 0 '4' and 7 '7'

Total time elapsed 0.0019991397857666016


'7777777'

## v5 Mathematical approach - No limit (Non recursive) - Optimized

So here it is, the final version. <br>

Exponential search has been implemented so let's talk about it. Why is it here ? <br>
I've figured out that v4 had a big flaw : it calculates every possibilities for the current space remaining for each possible value between 0 and space and stores each result. Then, it looks where is the break point (the index in the array where we pass over a certain value) and uses it in the algotihm. <br>
But imagine, if the break point is at index 0 and that the space is currently 1000 it will calculate the range from 0 to 999 and then find the break point. This is a huge flaw, because there are 999 values that have been calculated for nothing.<br>
So I have created an exponential search in non recursive to limit the amount of array access. Now it selects an index and then calculate the value of this index. It is faster because I do the minimum of array access and I don't calculate things for nothing.<br>

Reversed has been also implemented (it doesn't change much the result, it was just for the glory)<br>

Basically, everything has been implemented and I don't really see where I could optimize the algorithm even more.<br>
I'm really proud of this journey that comes to his end. It's been fun solving this problem.

Oh ? You're curious by how much the algorithm is faster than the v4 ? <br>
Well, when I tried with v4, 10^1000 took 4 hours to complete. Now, it takes 20 seconds. <br>
Yeah.

Is there anything I want to do ? Not really. Thanks for reading.

In [ ]:
import scipy.special
import time
from sys import stdout

class lucky_v5():
    def __init__(self, verbose=False):
        self.verbose = verbose
        self.start_time = 0
        
        self.query = 0
        self.cursor = 0
        self.length = 0
        self.limit = 0
        self.pair = []
        self.seven_amount = 0
        self.reverse = False
        self.total = 0
        self.solution = ""
        
    def verbose_print(self, text_index): #ignore this, it's just display
        if self.verbose == False:
            return
                                        #stdout.write is able to print on the same line and replace previous print
        if text_index == 0:
            self.start_time = time.time()
            print("Algorithm started")
        if text_index == 1:
            stdout.write("\rSolution will have a length of %d. Current length of amount of possibilities for this length is %d" \
                % (self.length, len(str(self.total))))
        if text_index == 2:
            print("\n")
            if self.reverse == 1:
                print("Number will be reversed")
            print("Now computing solution for query", self.query)
            print()
        if text_index == 3:
            stdout.write("\rCursor currently at position %d/%d (%d%%). Amount of seven put %d. Time elapsed %f" \
                % (self.limit-self.space, self.limit, int((self.limit-self.space)/self.limit*100), self.seven_amount, time.time() - self.start_time))
        if text_index == 4:
            print("\n")
            print("Algorithm finished")
            print("Solution have a length of", len(self.solution), "and have", self.solution.count("4"), "'4' and", \
                  self.solution.count("7"), "'7'")
            print()
            print("Total time elapsed", time.time() - self.start_time)
    
    def inverted(self, val): #to use the self.reverse function. Helps to write a generic algorithm
        if self.reverse == True:
            if val == "4":
                return "7"
            return "4"
        return val
    
    def is_lucky(self, number): #return 1 if number is a lucky number
        if any(x in str(number) for x in "01235689"):
            return 0
        return 1
    
    def get_palindrome_pair(self, length): #return amount of 4 and 7 that one side of a SLP can have
        pair = []
        is_odd = length % 2 #is_odd represent the possible middle digit countained in a odd length palindrome
        limit = length // 2 #we will simulate only one part of the palindrome because that's how the algorithm works
        total = 0
        for x in range(0, limit+1):
            if self.is_lucky(x*2) == 1 or self.is_lucky(x*2+is_odd) == 1 or \
            self.is_lucky(length-x*2) == 1 or self.is_lucky(length-x*2-is_odd) == 1:
                pair.append(x)
        return pair
    
    def get_comb(self, length, offset=0): #determine how much possibility for a given length
        total = 0                        #offset helps to make something more generic
        for i in self.pair:
            i -= offset
            if i <= length:
                total += scipy.special.comb(length, i, exact=True) #exact=True return a int instead of a float
            else:
                break
        return total
    
    def get_lucky_from_index(self, index): #get lucky number from index. "4" is index 0, "7" is 1 and so on
        binary = "{0:b}".format(index+2)   #I use binary because 4 and 7 are like 0 and 1
        res = ""
        for idx, i in enumerate(binary[1:]):
            res += "4" if i == "0" else "7"
        return int(res)
    
    def exponential_search(self, length, target):
        idx = 0
        step = 1 #helps moving on the array 
        current = -1 #store current value
        best_idx = -1 #return idx
        best_val = 0 #return value
        over_it = False #once we've got over the target we will change mult value

        while step > 0: #while we can move
            if over_it == True:
                step = step // 2 #need to reduce the steps to get precise

            if idx < length:
                current = self.get_comb(idx+1, self.seven_amount) #get the array[idx] value
            else:
                current = target + 1 #doing this we can simulate an infinite array. It makes smth generic easily

            if current <= target: #we update the best values because we're sure they are the current best
                best_val = current
                best_idx = idx
                idx += step
            else:
                if over_it == False: #little exception when breaking the point for the first time
                    step = step // 4
                    over_it = True #we know we crossed the line so now we need to get precise
                idx -= step

            if over_it == False:
                step *= 2       #need to increase the step to go further and faster

        return best_idx+1, best_val
    
    def slp_algorithm(self, init):
        self.cursor = self.query
        self.space = self.limit
        self.solution = init
        idx = 0
        remove = 1
        
        while self.cursor > 0 and remove > 0:
            self.verbose_print(3)
            idx, remove = self.exponential_search(self.space, self.cursor)
            self.solution[idx] = self.inverted("7")
            self.seven_amount += 1
            
            if self.pair[0] - self.seven_amount < 0: #slightly faster if we delete useless part
                del self.pair[0]
            self.cursor -= remove
            self.space = idx
        
        if self.cursor == 0:
            for i in range(self.pair[0] - self.seven_amount):
                self.solution[i] = self.inverted("7")
                self.seven_amount += 1
        self.space = 0
        self.verbose_print(3)
        
        for i in range(len(self.solution)):
            self.solution[i] = self.inverted(self.solution[i])
        
        return "".join(self.solution)
    
    def get_solution(self, query): #let's go
        self.verbose_print(0)
        
        tmp = 0
        self.query = query
        index = 0
        while self.query > 0: #we need to know the length of the solution
            self.length = self.get_lucky_from_index(index) #we first get the length of out current lucky number
            self.pair = self.get_palindrome_pair(self.length) #we determine the pair numbers of the length
            self.limit = self.length // 2
            self.total = self.get_comb(self.limit) #we calculate how much possibilities exist
            self.query -= self.total #if query > 0 that means the solution is not in this length
            index += 1
            self.verbose_print(1)
        self.query += self.total - 1       #need to go back one step to know what is the real query
        
        self.reverse = False    #used to helps algorithm by a little bit
        if (query - self.total//2) > 0:
            query = self.total - query + 1
            self.reverse = True
        
        self.seven_amount = 0 #we assume that the solution is filled by 4 where some of 4s will be replaced by 7s
        self.verbose_print(2)
        
        if self.reverse == False:
            self.solution = self.slp_algorithm(["4"] * self.limit) #everything
        else:
            self.solution = self.slp_algorithm(["7"] * self.limit) #everything
        
        if self.length%2 == 0: #we create the whole number from one side of it
            self.solution = self.solution[::-1] + self.solution
        else:                                        #and here we determine what is the value of the middle digit
            if self.is_lucky(self.seven_amount*2+1) == 1 or self.is_lucky((self.limit-self.seven_amount)*2) == 1:
                self.solution = self.solution[::-1] + "7" + self.solution
            else:
                self.solution = self.solution[::-1] + "4" + self.solution
        
        self.verbose_print(4)
        return self.solution
    

In [ ]:
for i in range(1, 10):
    
    print(i)
    res_v4 = lucky_v4().get_solution(i)
    res_v5 = lucky_v5().get_solution(i)
    if res_v4 != res_v5:
        print(res_v4)
        print(res_v5)

1
2
3
4
5
6
7
8
9


## Time comparison

Here you'll find a little script that will execute every algorithm and will display the amount of time each algorithm take to find the solution for different index.<br>
Indexes are selected for a good comparison between each algorithm.

In the second part of the cells you'll have a better difference between v4 and v5.<br>
You'll see that time depends mostly of the length and not really about the query

v3 is not compared because algorithm is false for big length number

In [ ]:
import time

v1 = lucky_v1()
v2 = lucky_v2()
v3 = lucky_v3()
v4 = lucky_v4()
v5 = lucky_v5()

for i in [50, 250, 2500, 10**5, 10**6]:
    res = []
    print("computing query", i)
    for v, algo in enumerate([v1, v2, v3, v4, v5]):
        start = time.time()
        res.append(algo.get_solution(i))
        print("v"+str(v+1) + " in " + str(int((time.time()-start)*1000)) + " ms")
    if len(set(res)) == 1:
        print("Solutions verified OK")
    else:
        print("Difference in solutions has been found")
    print()

print("------------------\n")

for i in [10, 11, 44, 45, 114, 115]: #power of ten
    res = []
    print("computing query 10^"+str(i))
    for v, algo in enumerate([v4, v5]):
        start = time.time()
        res.append(algo.get_solution(10**i))
        print("v"+str(v+4) + " in " + str(int((time.time()-start)*1000)) + " ms")
    if len(set(res)) == 1:
        print("Solutions verified OK")
    else:
        print("Difference in solutions has been found")
    print()

computing query 50
v1 in 0 ms
v2 in 1 ms
v3 in 0 ms
v4 in 0 ms
v5 in 0 ms
Solutions verified OK

computing query 250
v1 in 3036 ms
v2 in 2 ms
v3 in 0 ms
v4 in 0 ms
v5 in 0 ms
Solutions verified OK

computing query 2500
v1 in 6358 ms
v2 in 24 ms
v3 in 1 ms
v4 in 0 ms
v5 in 0 ms
Solutions verified OK

computing query 100000
v1 in 4967 ms
v2 in 820 ms
v3 in 0 ms
v4 in 1 ms
v5 in 0 ms
Solutions verified OK

computing query 1000000
v1 in 22169 ms
v2 in 7482 ms
v3 in 0 ms
v4 in 1 ms
v5 in 0 ms
Solutions verified OK

------------------

computing query 10^10
v4 in 1 ms
v5 in 0 ms
Solutions verified OK

computing query 10^11
v4 in 30 ms
v5 in 2 ms
Solutions verified OK

computing query 10^44
v4 in 132 ms
v5 in 6 ms
Solutions verified OK

computing query 10^45
v4 in 1765 ms
v5 in 11 ms
Solutions verified OK

computing query 10^114
v4 in 5293 ms
v5 in 76 ms
Solutions verified OK

computing query 10^115
v4 in 90424 ms
v5 in 51 ms
Solutions verified OK



### Will you dare to start it ? That's the final challenge
it will take a while tho

In [ ]:
print(lucky_v5(True).get_solution(10**10000))

Algorithm started
Solution will have a length of 74444. Current length of amount of possibilities for this length is 10897

Now computing solution for query 999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999

Cursor currently at position 5281/37222 (14%). Amount of seven put 715. Time elapsed 7840.724989